%load_ext autoreload
%autoreload 2

In [217]:
import pandas as pd
import numpy as np
import sys
sys.path.append('./')
import xiadan_check
import myModule
import imp
imp.reload(xiadan_check)
imp.reload(myModule)
import time

In [218]:
pathway = './'
all_files  = xiadan_check.get_files(pathway, "个性化","20181022")   ##给路径
df1  = xiadan_check.read_xlsx(all_files , 0 )## read all files sheet1

In [219]:
import datetime
def changetype1(check_str):
    value = 1 if type(check_str) == datetime.datetime else 0
    return value
def changetype2(check_str):
    value = 0 if type(check_str) == datetime.datetime else 1
    return value

In [220]:
date = pd.to_datetime(time.strftime('%Y-%m-%d',time.localtime(time.time())))
stat_week = '10月2周'
## df1:原始项目；
## df2:有效反馈项目，即排除驳回、暂停、沟通中
## df3:在线项目，基于df2
## df4:完成项目，基于df2
## df5:延期项目，基于df3中未交付延期、df4

myModule.add_week(df1,'下单时间',4)                                              #以下单时间定义week_number
myModule.upgrade_df(df1, [['反馈周期', ['评估反馈时间','下单时间'], '-']])     #增加“反馈周期”列
tt = lambda x : getattr(x , 'days')                                            #更换“days”类型
[myModule.retype(df1, i , tt) for i in ['反馈周期']] 

df2 = df1[df1['预计交付时间'].apply(lambda x: type(x) == datetime.datetime)]   #根据“预计交付时间”筛选有效反馈项目列表
df2['预计交付时间'] = df2['预计交付时间'].apply(pd.to_datetime)                #转换时间类型
myModule.upgrade_df(df2, [['预计周期', ['预计交付时间','评估反馈时间'], '-']]) #增加“预计周期”列
[myModule.retype(df2, i , tt) for i in ['预计周期']]                           #更换“days”类型

df3 = df2[df2['实际交付时间'].apply(lambda x: x == 0)]                          #根据“实际交付时间”筛选在线项目列表

df4 = df2[df2['实际交付时间'].apply(lambda x: type(x) == datetime.datetime)]    #根据“实际交付时间”筛选已交付项目列表
df4['实际交付时间'] = df4['实际交付时间'].apply(pd.to_datetime)                 #转换时间类型
myModule.upgrade_df(df4, [['实际周期', ['实际交付时间','评估反馈时间'], '-'],   #增加“实际周期”、“延期天数”列
                         ['延期天数',['实际交付时间','预计交付时间'], '-']])
[myModule.retype(df4, i , tt) for i in ['实际周期','延期天数']]                 #更换“days”类型
myModule.add_week(df4,'实际交付时间',4)                                           #以实际交付时间定义week_number

tmp = df3[df3['预计交付时间'].apply(lambda x: date>x)]                          #未交付延期项目
tmp['实际周期'] = 1000
tmp['延期天数'] = 1000
df5 = df4.append(tmp)
myModule.add_week(df5,'预计交付时间',4)                                           #以预计交付时间定义week_number

## 分析组人员列表
fenxi = ["施毓芷","闻少楠","杨宇","刘永强","刘梦苑","何胜","陈碧雯","王霞","卢楠","冯敬岩","朱慧萍","赵倩","解飞","宋少宇","卢楠"]

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:129: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

# 评估情况统计

In [221]:
df1.tail()

,序号,流水号,个性化分析名称,项目管理,接入时间,下单时间,信息负责人,评估反馈时间,预计交付时间,数据迁回花费时间（天）,实际交付时间,评估人时（h）,评估机时(h),是否在库里,库里周期（天）,项目管理交付时间,备注,week_number,反馈周期
323,1286,20181018-008,HA170069-02河南农业大学2个泡桐样本构建12个hic文库只分析任务单-VIP-02,王含,2018-10-18 00:00:00,2018-10-18,何胜,2018-10-19,2018-10-26 00:00:00,0,0,0,0.0,0,0,0,0.0,10月2周,1
324,1287,20181018-009,HA170069-02河南农业大学2个泡桐样本构建12个hic文库只分析任务单-VIP-03,王含,2018-10-18 00:00:00,2018-10-18,驳回,2018-10-18,驳回,0,0,0,0.0,0,0,0,0.0,10月2周,0
325,1288,20181018-010,ANCJS170153南通大学300个大鼠神经元单细胞lncRNA建库测序分析任务单-VIP-07,廖星吕,2018-10-18 00:00:00,2018-10-18,赵倩,2018-10-19,2018-10-25 00:00:00,0,0,0,0.0,0,0,0,0.0,10月2周,1
326,1289,20181018-011,BJ170435-01动物所12个小鼠MALBAC扩增单细胞基因组建库测序分析任务单_VIP-13,廖星吕,2018-10-18 00:00:00,2018-10-18,刘江,2018-10-18,2018-10-28 00:00:00,0,0,0,0.0,0,0,0,0.0,10月2周,0
327,1290,20181018-012,BJ170435-01动物所12个小鼠MALBAC扩增单细胞基因组建库测序分析任务单_VIP-14,廖星吕,2018-10-18 00:00:00,2018-10-18,刘江,2018-10-18,2018-10-28 00:00:00,0,0,0,0.0,0,0,0,0.0,10月2周,0


## 接入项目数统计

### 整体接入项目数趋势图

In [222]:
ss = myModule.format(df1,'week_number',['流水号','个性化分析名称','项目管理','预计交付时间'])
ss.df['接入项目数'] = list(map(changetype2,ss.df['流水号']))
ss.df['有效接入项目数'] = list(map(changetype1,ss.df['预计交付时间']))
ss.df['驳回暂停沟通项目数'] = list(map(changetype2,ss.df['预计交付时间']))
tt = myModule.format(ss.df,'week_number',['有效接入项目数','驳回暂停沟通项目数','接入项目数'])
pp = myModule.sort_by_index(tt.groupby('sum'))
myModule.myPlot(pp).plot('line')

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

### 个人接入项目数分布图

In [223]:
ss = myModule.format(df1 ,'week_number' ,['流水号'])
ss.add_group('信息负责人')
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('stack_bar')
#myModule.get_rank(ss.df , '信息负责人' ,'流水号', quantile=0.8, compare='>' )

## 评估周期情况统计

### 预计周期趋势图

In [224]:
# 平均评估周期，建议添加最长评估周期、25%、50%、80%项目评估周期的指标
ss = myModule.format(df2 ,'week_number' ,['预计周期'])
pp = myModule.sort_by_index(ss.groupby('mean'))
myModule.myPlot(pp).plot('line')

### 预计周期>7日项目列表

In [225]:
ss = myModule.format(df2,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','预计交付时间','预计周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('预计周期',7)
ss.select_by_value('预计周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','预计交付时间','预计周期']).df
tt.sort_values(by = '预计周期',axis = 0,ascending = False)   #预计周期从大到小排序

,week_number,流水号,个性化分析名称,信息负责人,下单时间,评估反馈时间,预计交付时间,预计周期
308,10月2周,20181017-012,PM-HB180225-01华中农大1个白背飞虱Hi-C辅助基因组组装建库测序分析任务单-02,占伟,2018-10-17,2018-10-18,2018-11-02,15
293,10月2周,20181016-012,SN170074-02陕西师范大学1个黄芩基因组三代数据组装进化只分析任务单-05,杨伟飞,2018-10-16,2018-10-16,2018-10-29,13
252,10月2周,20181011-011,BJ181222-02遗传所1个植物全长转录组只分析任务单-02,刘永强,2018-10-12,2018-10-18,2018-10-29,11
326,10月2周,20181018-011,BJ170435-01动物所12个小鼠MALBAC扩增单细胞基因组建库测序分析任务单_VIP-13,刘江,2018-10-18,2018-10-18,2018-10-28,10
327,10月2周,20181018-012,BJ170435-01动物所12个小鼠MALBAC扩增单细胞基因组建库测序分析任务单_VIP-14,刘江,2018-10-18,2018-10-18,2018-10-28,10
304,10月2周,20181017-008,ANCBJ170753协和医院2个单细胞转录组10x genomics测序分析-04,姚盟成,2018-10-17,2018-10-18,2018-10-26,8
305,10月2周,20181017-009,ANCBJ170753协和医院2个单细胞转录组10x genomics测序分析-05,姚盟成,2018-10-17,2018-10-18,2018-10-26,8
314,10月2周,20181017-018,PM-GD180505-01华农4个青枯菌hic建库测序分析任务单-02,昶吉阳,2018-10-18,2018-10-22,2018-10-30,8


## 反馈延期情况统计

### 反馈延期率趋势图

In [226]:
ss = myModule.format(df1 ,'week_number' ,['流水号','反馈周期'])
ss.judge_and_replace('反馈周期',1)
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['反馈周期_bool'] = ss.df['反馈周期_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['反馈延期率' ,'反馈周期_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:249: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

### 反馈延期项目列表

In [227]:
ss = myModule.format(df1,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','反馈周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('反馈周期',1)
ss.select_by_value('反馈周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','下单时间','评估反馈时间','反馈周期']).df
tt.sort_values(by = '反馈周期',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,下单时间,评估反馈时间,反馈周期
252,10月2周,20181011-011,BJ181222-02遗传所1个植物全长转录组只分析任务单-02,刘永强,2018-10-12,2018-10-18,6
321,10月2周,20181018-006,PM-BJ181076-01基因组所2个细菌HiC建库测序任务单-01,屈昂,2018-10-18,2018-10-23,5
314,10月2周,20181017-018,PM-GD180505-01华农4个青枯菌hic建库测序分析任务单-02,昶吉阳,2018-10-18,2018-10-22,4


## 在线项目数统计

In [228]:
ss = myModule.format(df3 ,'信息负责人' ,['流水号'])
pp = ss.groupby('count')
myModule.myPlot(pp).plot('bar')

# 交付情况统计

## 完成项目数统计

### 整体完成项目数趋势图

In [229]:
ss = myModule.format(df4 ,'week_number' ,['实际交付时间'])
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('line')

### 个人完成项目数统计

In [230]:
ss = myModule.format(df4 ,'week_number' ,['实际交付时间'])
ss.add_group('信息负责人')
pp = myModule.sort_by_index(ss.groupby('count'))
myModule.myPlot(pp).plot('stack_bar')

## 完成周期统计

### 整体完成周期趋势图

In [231]:
# 实际完成平均周期，建议添加最长完成周期、25%、50%、80%项目完成周期的指标
ss = myModule.format(df4 ,'week_number' ,['实际周期'])
pp = myModule.sort_by_index(ss.groupby('mean'))
myModule.myPlot(pp).plot('line')

### 个人完成周期统计

In [232]:
df4_fx = df4[df4['信息负责人'].isin(fenxi)] 
ss = myModule.format(df4_fx,'信息负责人' ,['实际周期'])
pp = ss.groupby('mean')
myModule.myPlot(pp).plot('bar')

# 延期情况统计

## 延期率趋势图

In [233]:
ss = myModule.format(df5 ,'week_number' ,['流水号','延期天数'])
ss.judge_and_replace('延期天数',1)
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['延期天数_bool'] = ss.df['延期天数_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['项目延期率' ,'延期天数_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:249: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

## 延期项目列表

In [234]:
ss = myModule.format(df5,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间','延期天数'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('延期天数',1)
ss.select_by_value('延期天数_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','预计交付时间','实际交付时间']).df
tt.sort_values(by = '预计交付时间',axis = 0,ascending = False)

,week_number,流水号,个性化分析名称,信息负责人,评估反馈时间,预计交付时间,实际交付时间
237,10月2周,20181010-010,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-03,宋少宇,2018-10-11,2018-10-18,0
238,10月2周,20181010-011,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-04,宋少宇,2018-10-11,2018-10-18,0
239,10月2周,20181010-012,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-05,宋少宇,2018-10-11,2018-10-18,0
279,10月2周,20181015-008,PM-BJ181079-03动物所9个小鼠Bisulfite甲基化建库测序分析任务单-05,韩少怀,2018-10-16,2018-10-17,0
190,10月2周,20180929-005,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-01,宋少宇,2018-10-08,2018-10-16,0
195,10月2周,20180930-003,PM-SD180234-01山东省立医院78个人肿瘤外显子建库测序自动化分析任务单-01,封彦杰,2018-10-08,2018-10-16,0
207,10月2周,20181008-010,PM-GD170317-01深圳市人民医院16个人6mA建库测序分析任务单-02,宋少宇,2018-10-09,2018-10-16,0
156,10月2周,20180925-007,PM-SD170191-02山东大学生殖医学研究中心1个人hic文库测序分析任务单-VIP-02,范旭宁,2018-09-26,2018-10-15,0


# 7日完成率统计

##  7日完成率趋势图

In [235]:
ss = myModule.format(df4 ,'week_number' ,['流水号','实际周期'])
ss.judge_and_replace('实际周期',7)   # 建议添加小于号
ss.df['流水号'] = list(map(changetype2, ss.df['流水号']))
ss.df['实际周期_bool'] = ss.df['实际周期_bool'].astype(int)      #bool类型装换为数值1/0
pp = myModule.sort_by_index(ss.get_total_ratio([['7日完成率' ,'实际周期_bool' , '流水号','/']]))
myModule.myPlot(pp).plot('line')

/home/jiankong/jupyter/chart/bin/supplment_analysis/myModule.py:249: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/jiankong/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

## 实际周期>7日项目列表

In [236]:
ss = myModule.format(df4,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','实际交付时间','实际周期'])
ss.select_by_value('week_number',stat_week)
ss.judge_and_replace('实际周期',7)
ss.select_by_value('实际周期_bool',True)
tt = myModule.format(ss.df,'week_number',['流水号','个性化分析名称','信息负责人','评估反馈时间','实际交付时间','实际周期']).df
tt.sort_values(by = '实际周期',axis = 0,ascending = False)   #预计周期从大到小排序

,week_number,流水号,个性化分析名称,信息负责人,评估反馈时间,实际交付时间,实际周期
124,10月2周,20180920-003,PM-BJ170057-01、02/03北京蛋白质组研究中心4个小鼠10XGenomics文...,姚盟成,2018-09-20,2018-10-12,22
178,10月2周,20180927-006,PM-ZJ180327-01浙医二院35个人普通转录组建库测序过滤任务单-快速产品-01,屈昂,2018-09-30,2018-10-16,16
194,10月2周,20180930-002,PM-GD180479-01中山大学6个人有参普通转录组建库测序分析任务单-快速产品-02,杨宇,2018-09-30,2018-10-15,15
179,10月2周,20180927-007,BJ180998-02中药所1个药用动物S4基因组denovo只分析任务单,马潇,2018-09-28,2018-10-12,14
193,10月2周,20180930-001,SD170077-01山东大学齐鲁医院王刚的6个人单基因病外显子建库测序分析任务单-01,解飞,2018-09-30,2018-10-12,12
198,10月2周,20181008-001,PM-YWHB180262-01华科协和医院2个人全基因组重测序建库测序分析任务单-快速产品-01,车天栋,2018-10-08,2018-10-18,10
203,10月2周,20181008-006,PM-BJ180807-01医科院药物所15个人有参转录组建库测序分析任务单-01,冯敬岩,2018-10-09,2018-10-18,9
217,10月2周,20181009-009,PM-YWJS180225-01徐州市中心医院20个大鼠粪便宏基因组建库测序过滤任务单-快速产品,冷雪,2018-10-09,2018-10-18,9
221,10月2周,20181009-013,SD170174-04山东农大1个芒属植物Hi-C辅助基因组组装只分析任务单,刘江,2018-10-09,2018-10-17,8
234,10月2周,20181010-007,PM-BJ181079-03动物所9个小鼠Bisulfite甲基化建库测序分析任务单-04,韩少怀,2018-10-10,2018-10-18,8
